# Contextual AI Reranker Integration with DSPy

This tutorial demonstrates how to integrate Contextual AI's instruction-following reranker with DSPy for improved retrieval-augmented generation (RAG) performance. We'll show how DSPy's optimization capabilities can work with Contextual AI's reranking to achieve better results.


## Setup


In [ ]:
import dspy
import os
import requests
import ujson
import random
from dspy.utils import download
from dspy.evaluate import SemanticF1

# Set API keys
# Get this key at http://app.contextual.ai/
CONTEXTUAL_API_KEY = "your_contextual_api_key"
OPENAI_API_KEY = "your_openai_api_key"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

lm = dspy.LM(model="openai/gpt-4o-mini", api_key=OPENAI_API_KEY)
dspy.settings.configure(lm=lm)


## Contextual AI Reranker Implementation


In [7]:
class ContextualReranker(dspy.Retrieve):
    def __init__(self, api_key, base_retriever=None, k=5, rerank_instructions=""):
        super().__init__(k=k)
        self.api_key = api_key
        self.base_retriever = base_retriever
        self.rerank_instructions = rerank_instructions
    
    def forward(self, query_or_queries):
        if self.base_retriever:
            # Get initial documents from base retriever
            initial_docs = self.base_retriever(query_or_queries)
            documents = initial_docs.passages
        else:
            documents = query_or_queries if isinstance(query_or_queries, list) else [query_or_queries]
        
        url = "https://api.contextual.ai/v1/rerank"
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": f"Bearer {self.api_key}"
        }
        
        payload = {
            "query": query_or_queries if isinstance(query_or_queries, str) else query_or_queries[0],
            "documents": documents,
            "model": "ctxl-rerank-v2-instruct-multilingual",
            "top_n": self.k,
            "instruction": self.rerank_instructions
        }
        
        response = requests.post(url, headers=headers, json=payload)
        result = response.json()
        
        if 'results' not in result:
            print("Contextual API Error:", result)
            return dspy.Prediction(passages=documents[:self.k])
        
        reranked_results = result['results']
        reranked_results.sort(key=lambda x: x['relevance_score'], reverse=True)
        
        top_docs = []
        for item in reranked_results[:self.k]:
            doc = documents[item['index']]
            if isinstance(doc, str):
                top_docs.append(doc)
            else:
                top_docs.append(doc.get('content', doc))
        
        return dspy.Prediction(passages=top_docs)


## Load Dataset


In [9]:
# Download and load the RAG-QA Arena Tech dataset
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl")

with open("ragqa_arena_tech_corpus.jsonl") as f:
    corpus_lines = f.readlines()[:100]

corpus = []
for line in corpus_lines:
    data = ujson.loads(line)
    corpus.append(data['text'])

print(f"Loaded {len(corpus)} documents")


Loaded 100 documents


In [10]:
# Load question-answer pairs for evaluation
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl")

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

data = [dspy.Example(**d).with_inputs('question') for d in data]

# Split data for training, validation, and testing
random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

print(f"Training: {len(trainset)}, Dev: {len(devset)}, Test: {len(testset)}")


Training: 200, Dev: 300, Test: 500


## Setup Base Retriever


In [11]:
# Create base embedding retriever
embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512, api_key=OPENAI_API_KEY)
base_search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=10)


## Create RAG Systems


In [ ]:
class RAG(dspy.Module):
    def __init__(self, retriever):
        super().__init__()
        self.retriever = retriever
        self.respond = dspy.ChainOfThought('context, question -> response')
    
    def forward(self, question):
        if hasattr(question, 'question'):
            question = question.question
        elif isinstance(question, dict) and 'question' in question:
            question = question['question']
        context = self.retriever(question).passages
        return self.respond(context=context, question=question)

# Create different RAG systems
base_rag = RAG(base_search)

# RAG with Contextual AI reranking -> Changing the prompt can yield different results
contextual_reranker = ContextualReranker(
    api_key=CONTEXTUAL_API_KEY,
    base_retriever=base_search,
    k=5,
    rerank_instructions="Prioritize documents that provide specific, actionable technical solutions and step-by-step instructions."
)
reranked_rag = RAG(contextual_reranker)


## Evaluation Setup


In [13]:
# Setup evaluation metric
metric = SemanticF1(decompositional=True)
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=4, display_progress=True)


## Baseline Evaluation


## Test the Reranker

Let's test our reranker with a simple example to see how it works.


In [16]:
# Test query - something relevant to the tech corpus
test_query = "How do I fix my Linux system"

print(f"Query: {test_query}\n")

base_results = base_search(test_query)
print("Base retrieval results:")
for i, doc in enumerate(base_results.passages[:3]):
    print(f"{i+1}. {doc[:200]}...\n")

reranked_results = contextual_reranker(test_query)
print("Reranked results:")
for i, doc in enumerate(reranked_results.passages[:3]):
    print(f"{i+1}. {doc[:200]}...\n")


Query: How do I fix my Linux system

Base retrieval results:
1. My resolution was similar to Roman Ts however I needed to add a few extra steps. In my case I had Ubuntu Server 14 VM running on a Windows 8 Desktop in Windows 2008 domain. If I tried NAT or Bridge I ...

2. You can, but its a major security and stability risk. Doing so allows any application full access to your computer. You cant know what theyre doing with that access. Its unnecessary, and just really u...

3. Android shares very little with a typical Linux distribution. In fact, this is where Richard Stallmans GNU/Linux distinction comes in handy — Android isnt really a Unix-like general purpose operating ...

Reranked results:
1. My resolution was similar to Roman Ts however I needed to add a few extra steps. In my case I had Ubuntu Server 14 VM running on a Windows 8 Desktop in Windows 2008 domain. If I tried NAT or Bridge I ...

2. The primary command to manipulate deb packages is dpkg-deb. To unpack the package, cre

In [ ]:
# Same Query
base_response = base_rag(question=test_query)
reranked_response = reranked_rag(question=test_query)

print(f"Base RAG Response:{base_response.response}")


Base RAG Response:To address issues with your Linux system, especially related to networking or SSH access, follow these steps:

1. **Network Configuration**: Check your network adapters' configuration. If you're using a VM, ensure you have one adapter set to Bridged for Internet access and another set to Host-Only for local communication.

2. **Edit Configuration Files**: If you're not getting an IP address, express the network configuration by editing the `/etc/network/interfaces` file. Add the following lines for your second interface (e.g., eth1):
   ```
   auto eth1
   iface eth1 inet dhcp
   ```
   Save the file and restart the networking service:
   ```
   sudo service networking restart
   ```
   or restart the VM if that does not work.

3. **Test SSH Access**: After modifying settings, use `ifconfig -a` to check if the interface has received an IP address and attempt to SSH into the VM.

4. **Package Issues**: If you are experiencing software-related issues, utilize package ma

In [18]:
print(f"Reranked RAG Response:{reranked_response.response}")

Reranked RAG Response:To address issues with your Linux system:

1. **For Networking Issues:**
   - Ensure you have the correct network adapter settings.
   - You can add a second network adapter (bridged and host-only) as described in the context.
   - Edit `/etc/network/interfaces` to configure DHCP for each adapter.
   - Restart the networking service or the entire VM to ensure changes take effect.
   - Use `ifconfig -a` to verify if you have obtained an IP address.

2. **For Package Management Problems:**
   - If you're having trouble with .deb files, utilize `dpkg-deb` to unpack and later rebuild packages. Make sure to run commands using `fakeroot` if you need to handle permissions without root access.
   - Modify the control files carefully and increment version numbers if necessary to avoid conflicts.

If you provide more specific information about the issue you're encountering, I can give more tailored advice.


In [14]:
print("Evaluating Contextual AI reranked RAG system...")
reranked_score = evaluate(reranked_rag)


Evaluating Contextual AI reranked RAG system...
Average Metric: 122.68 / 300 (40.9%): 100%|██████████| 300/300 [18:16<00:00,  3.66s/it]

2025/10/09 01:00:04 INFO dspy.evaluate.evaluate: Average Metric: 122.67520181732142 / 300 (40.9%)


> **Note:**  
> This notebook is primarily an integration demo, not a performance benchmark.  
> but demonstrates how to extend DSPy with Contextual's reranking APIs.